In [ ]:
import cv2
import numpy as np
import threading
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, widgets

In [ ]:

# 初始分辨率设置
width, height = 720, 480


In [ ]:
button_layout = widgets.Layout(width='200px', height='50px', align_self='center')
output = widgets.Output()
# 退出控件
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
# 图像控件
imgbox = widgets.Image(format='jpg', width=width*2,  height=height*2,layout=widgets.Layout(align_self='center'))
# 空间布局
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

def exit_button_Callback(value):
    global model
    model = 'Exit'
exit_button.on_click(exit_button_Callback)

# 控制模型状态
model = 'General'


In [ ]:
# 构建通道字符串
def build_pipeline(sensor_id):
    width = 720
    height = 640
    framerate = 30
    pipeline = (
        f"nvarguscamerasrc sensor-id={sensor_id} ! "
        f"video/x-raw(memory:NVMM), width={width}, height={height}, format=(string)NV12, framerate=(fraction){framerate}/1 ! "
        "nvvidconv ! "
        "video/x-raw, format=(string)BGRx ! "
        "videoconvert ! "
        "appsink"
    )
    return pipeline


# 图像捕获函数
def capture_frames():
    # 打开GStreamer管道
    cap1 = cv2.VideoCapture(build_pipeline(0), cv2.CAP_GSTREAMER)
    cap2 = cv2.VideoCapture(build_pipeline(1), cv2.CAP_GSTREAMER)
    while True:
        try:
            # 读取帧
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            if (not ret1) and (not ret2):
                print("无法从nvarguscamerasrc读取帧")
                break

            if model == "Exit":
                cv2.destroyAllWindows()
                cap1.release()
                cap2.release()
                break

            # 水平拼接两个图像
            combined_image = cv2.hconcat([frame1, frame2])

            imgbox.value = cv2.imencode(".jpg", combined_image)[1].tobytes()
            interact(imgbox.value, i=(0, 2))

        except KeyboardInterrupt:
            cap1.release()
            cap2.release()


# 构建通道字符串
def build_pipeline(sensor_id, width=640, height=480, framerate=30):
    pipeline = (
        f"nvarguscamerasrc sensor-id={sensor_id} ! "
        f"video/x-raw(memory:NVMM), width={width}, height={height}, format=(string)NV12, framerate=(fraction){framerate}/1 ! "
        "nvvidconv ! "
        "video/x-raw, format=(string)BGRx ! "
        "videoconvert ! "
        "appsink"
    )
    return pipeline


# 根据提供的分辨率选项调整宽度和高度
resolution_options = {
    "640x480": (640, 480),
    "1280x720": (1280, 720),
    "1920x1080": (1920, 1080),
}


# 图像尺寸变更选项
def get_reszie(resolution):
    global width, height
    width, height = resolution_options[resolution]
    return width, height


# 调整交互函数，按比例调整分辨率
interact(
    get_reszie,
    resolution=widgets.Dropdown(
        options=resolution_options.keys(), description="Image Size: "
    ),
)


# 图像捕获函数
def capture_frames():
    # 打开GStreamer管道
    cap1 = cv2.VideoCapture(
        build_pipeline(sensor_id=0, width=width, height=height, framerate=30),
        cv2.CAP_GSTREAMER,
    )
    cap2 = cv2.VideoCapture(
        build_pipeline(sensor_id=1, width=width, height=height, framerate=30),
        cv2.CAP_GSTREAMER,
    )
    while True:
        try:
            # 读取帧
            ret1, frame1 = cap1.read()
            ret2, frame2 = cap2.read()
            if (not ret1) and (not ret2):
                print("无法从nvarguscamerasrc读取帧")
                break

            frame1 = cv2.resize(frame1, (width, height))
            frame2 = cv2.resize(frame2, (width, height))

            if model == "Exit":
                cv2.destroyAllWindows()
                cap1.release()
                cap2.release()
                break
            if frame1.shape == frame2.shape and frame1.dtype == frame2.dtype:
                # 水平拼接两个图像
                combined_image = np.hstack((frame1, frame2))
                # combined_image = cv2.hconcat([frame1, frame2])
                imgbox.value = cv2.imencode(".jpg", combined_image)[1].tobytes()
            else:
                print("Frames are not compatible for concatenation.")

        except KeyboardInterrupt:
            cap1.release()
            cap2.release()

In [ ]:

# 显示控件
display(controls_box,output)

# 在新线程中捕获图像
threading.Thread(target=capture_frames).start()
